<a href="https://colab.research.google.com/github/gee-community/geemap/blob/master/examples/notebooks/geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Using geemap with Google Colab

## Introduction

This notebook demonstrates how to use [geemap](https://geemap.org) with Google Colab. To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial).

## Install geemap

The geemap package has been pre-installed in Google Colab. However, if you would like to install the latest version, you can uncomment and run the following command:

In [ ]:
# %pip install -U geemap

## Import libraries

Import the Earth Engine Python API and geemap.

In [ ]:
import ee
import geemap

## EE Authentication

Run the following cell will start the Earth Engine authentication. Follow the instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication) to authenticate Earth Engine.

In [ ]:
geemap.ee_initialize()

## Core features

You can import the geemap package using the following conventions:

- To import only the core features of geemap, use: `import geemap.core as geemap`
- To import all the features of geemap, use: `import geemap`

Let's import the geemap package using the second option:

In [ ]:
import geemap.core as geemap

## Basemap selector

Select a basemap from the dropdown list and add it to the map.

In [ ]:
m = geemap.Map()
m.add("basemap_selector")
m

## Layer Manager

Toggle the checkbox to show or hide the layer. Drag and move the slider to change the transparency level of the layer.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add_layer(states, {}, "US States")
m.add('layer_manager')
m

## Inspector

Click on the map to query Earth Engine data at a specific location.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add_layer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
m.add_layer(states, {}, "US States")
m.add('inspector')
m

## Layer Editor

You can change the visualization parameters of the Earth Engine data using the layer editor.

### Single-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add('layer_editor', layer_dict=m.ee_layers['SRTM DEM'])
m

### Multi-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
m.add_layer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
m.add('layer_editor', layer_dict=m.ee_layers['Landsat 7'])
m

### Feature collection

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
m.add_layer(states, {}, "US States")
m.add('layer_editor', layer_dict=m.ee_layers['US States'])
m

## Draw control

You can draw shapes on the map using the draw control. The drawn features will be automatically converted to Earth Engine objects, which can be accessed in one of the following ways:

- To return the last drawn feature as an `ee.Geometry()`, use: `m._draw_control.last_geometry`
- To return all the drawn feature as an `ee.Feature()`, use: `m._draw_control.last_feature`
- To return all the drawn features as an `ee.FeatureCollection()`, use: `m._draw_control.collection`

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': 'terrain',
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add('layer_manager')
m

Use the draw control to draw a polygon on the map.

In [ ]:
geometry = m._draw_control.last_geometry
geometry

In [ ]:
feature = m._draw_control.last_feature
feature

In [ ]:
collection = m._draw_control.collection
collection

Clip the DEM data using the drawn polygon.

In [ ]:
if geometry is not None:
    image = dem.clipToCollection(collection)
    m.layers[1].visible = False
    m.add_layer(image, vis_params, "Clipped DEM")
m